In [ ]:
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

def read_metrics(file_path):
    metrics = pd.read_csv(file_path)
    metrics['total_time_us'] = metrics['total_time_us'] - metrics['inference_start']
    metrics['non_max_suppression'] = metrics['non_max_suppression'] - metrics['model']
    metrics['model'] = metrics['model'] - metrics['inference_start']

    metrics['inference_start'] = metrics['inference_start'] - metrics['inference_start'][0]

    metrics.iloc(1)[:4] = metrics.iloc(1)[:4] // 1000
    metrics = metrics[metrics['model'] < metrics['model'].quantile(0.99)]
    metrics['model_per_frame'] = metrics['model'] // metrics['batch_size']
    metrics['total_per_frame'] = metrics['total_time_us'] // metrics['batch_size']
    return metrics

def create_graph(metrics_list):
    fig = make_subplots(rows=len(metrics_list), cols=1, shared_xaxes=True, vertical_spacing=0.05)
    for idx, metric in enumerate(metrics_list):
        fig.add_histogram(x=metric['total_per_frame'], row=idx+1, col=1, nbinsx=200)
    fig.update_layout(height=150*len(metrics_list))
    return fig
    

In [ ]:
files = [
    'metrics_cpu_1.csv',
    'metrics_cpu_3.csv',
    'metrics_cpu_5.csv',
    'metrics_cpu_10.csv',
    'metrics_cpu_20.csv',
]

create_graph([read_metrics(f) for f in files]).show()


In [ ]:
files = [
    'metrics_gpu_5.csv',
    'metrics_gpu_10.csv',
    'metrics_gpu_20.csv',
    'metrics_gpu_30.csv',
    'metrics_gpu_50.csv',
]

create_graph([read_metrics(f) for f in files]).show()


In [ ]:
fig2 = px.scatter(metrics, x="inference_start", y="model")
fig2.show()